# Player Stats

In [4]:
from fotmob_api import FotmobAPI
import pandas as pd

# Initialize client
client = FotmobAPI()

# Get fixtures for a specific league
# get league id
all_leagues = client.get_league_all()
df_country_leagues = pd.json_normalize(all_leagues["countries"],
                                       record_path=["leagues"],
                                       meta=["ccode", "name", "localizedName"],
                                       record_prefix="league")
league_id = df_country_leagues[(df_country_leagues["leaguename"]=="Premier League") &
                               (df_country_leagues["name"]=="England")]["leagueid"].item()

# Get league table in order to get team ids
league_table = client.get_league_table(league_id=league_id)
team_ids = {team["name"]: team["id"] for team in league_table[0]["data"]["table"]["all"]}
team_ids

{'Manchester City': 8456,
 'Liverpool': 8650,
 'Arsenal': 9825,
 'Aston Villa': 10252,
 'Tottenham Hotspur': 8586,
 'Manchester United': 10260,
 'West Ham United': 8654,
 'Brighton & Hove Albion': 10204,
 'Newcastle United': 10261,
 'Wolverhampton Wanderers': 8602,
 'Chelsea': 8455,
 'AFC Bournemouth': 8678,
 'Fulham': 9879,
 'Crystal Palace': 9826,
 'Brentford': 9937,
 'Nottingham Forest': 10203,
 'Luton Town': 8346,
 'Everton': 8668,
 'Burnley': 8191,
 'Sheffield United': 8657}

In [5]:
# Get squad details for a team in order to get player id
team_name = "Liverpool"
team_info = client.get_team(team_id=[team_ids[team_name]])
df_squad = pd.concat([pd.DataFrame(squad[1]) for squad in team_info["squad"]])
df_squad.head()

,id,name,ccode,cname,role,isInjured
0,78660,Jürgen Klopp,GER,Germany,NaN,NaN
0,169756,Adrian,ESP,Spain,goalkeepers,NaN
1,319784,Alisson Becker,BRA,Brazil,goalkeepers,NaN
2,776689,Caoimhin Kelleher,IRL,Ireland,goalkeepers,NaN
0,171698,Joel Matip,CMR,Cameroon,defenders,True


In [6]:
# Get stats from a player
player_name = "Mohamed Salah"
player_id = df_squad[df_squad["name"] == player_name]["id"].item()

players_stats = client.get_player_stats(player_id=player_id, league_id=league_id, season="2023/2024")
df_player_stats = pd.json_normalize(players_stats["statsSection"],
                                    record_path=["items", ["items"]],
                                    meta="title", meta_prefix="meta_")
df_player_stats.head()

,title,localizedTitleId,statValue,per90,percentileRank,percentileRankPer90,statFormat,meta_title
0,Goals,goals,14,0.722892,100.000000,96.341463,number,Season performance
1,xG,expected_goals,14.27,0.737050,100.000000,100.000000,fraction,Season performance
2,xGOT,expected_goals_on_target,14.49,0.748327,100.000000,97.560976,fraction,Season performance
3,Penalty goals,goals_subtitle,4,0.206540,96.341463,93.902439,number,Season performance
4,xG excl. penalty,non_penalty_xg,9.54,0.492795,100.000000,96.341463,fraction,Season performance
